In [45]:
import pandas as pd
from datetime import datetime
import time
from tabulate import tabulate
import concurrent.futures

class StockTrader:
    def __init__(self, initial_balance=100000):
        self.balance = initial_balance
        self.stock_quantity = 0
        self.stock_price = 0
        self.last_generated_id = int(time.time())
        self.exit_reason = 'Manual'
        columns = ['Symbol', 'Status', 'BuyID', 'Quantity', 'BuyPrice', 'BuyDatetime', 'SellTransactions',
                   'PendingQuantity', 'AverageSellPrice', 'PnL', 'SLValue', 'TargetValue', 'ExitReason']
        self.logs = pd.DataFrame(columns=columns)
    def generate_id(self):
        unique_id = self.last_generated_id
        self.last_generated_id += 1
        return unique_id

    def buy_stock(self, quantity, price, date, symbol):
        buy_id = self.generate_id()
        buy_datetime = datetime.strptime(date, '%Y-%m-%d')
        self.balance -= quantity * price
        self.stock_quantity += quantity
        self.stock_price = price
        self.stoploss = price -10
        self.target = price + 20
        new_log = pd.DataFrame({'Symbol': [symbol], 'BuyID': [buy_id], 'Quantity': [quantity], 'BuyPrice': [price],
                                'BuyDatetime': [buy_datetime], 'SellTransactions': [[]], 'PendingQuantity': [quantity],
                                'AverageSellPrice': [0], 'PnL': [0], 'Status': ['Open'],
                                'SLValue': [self.stoploss], 'TargetValue': [self.target], 'ExitReason': [self.exit_reason]})
        self.logs = pd.concat([self.logs, new_log], ignore_index=True)
        return buy_id

    def sell_stock(self, buy_id, quantity, sell_price, date, exit_reason='Manual'):
        buy_row = self.logs[self.logs['BuyID'] == buy_id]
        if buy_row.empty:
            return "Buy ID not found."

        sell_datetime = datetime.strptime(date, '%Y-%m-%d')
        pnl = quantity * (sell_price - buy_row['BuyPrice'].values[0])

        # Update the log entry with the sell transaction
        self.balance += quantity * sell_price
        buy_index = buy_row.index[0]
        sell_transaction = {'Quantity': quantity, 'SellPrice': sell_price, 'SellDatetime': sell_datetime, 'PnL': pnl}
        self.logs.at[buy_index, 'SellTransactions'] = self.logs.at[buy_index, 'SellTransactions'] + [sell_transaction]

        # Update pending quantity, average sell price, and cumulative PnL
        self.logs.at[buy_index, 'PendingQuantity'] -= quantity
        self.logs.at[buy_index, 'AverageSellPrice'] = self.calculate_average_sell_price(buy_index)
        self.logs.at[buy_index, 'PnL'] += pnl

        # If all quantity sold, update the status to 'Done'
        if self.logs.at[buy_index, 'PendingQuantity'] == 0:
            self.logs.at[buy_index, 'Status'] = 'Done'

        # Set exit reason and update SLValue and TargetValue
        self.exit_reason = exit_reason
        self.logs.at[buy_index, 'SLValue'] = self.stoploss
        self.logs.at[buy_index, 'TargetValue'] = self.target
        self.logs.at[buy_index, 'ExitReason'] = exit_reason

        return f"Stock sold successfully. PnL: {pnl}"

    def calculate_average_sell_price(self, buy_index):
        sell_transactions = self.logs.at[buy_index, 'SellTransactions']
        if not sell_transactions:
            return 0

        total_sell_price = sum(transaction['Quantity'] * transaction['SellPrice'] for transaction in sell_transactions)
        total_quantity = sum(transaction['Quantity'] for transaction in sell_transactions)

        return total_sell_price / total_quantity

    def set_sl(self,stoploss = 10):
        self.stoploss = self.stock_price - stoploss
        self.logs.at[-1, 'SLValue'] = self.stoploss

    def set_target(self,target = 20):
        self.target = self.stock_price + target
        self.logs.at[-1, 'TargetValue'] = self.target

    def auto_exit(self, current_price):
        buy_index = self.logs.index[-1]
        qnty = self.logs.at[buy_index, 'PendingQuantity']
        if qnty != 0 :
            if current_price >= self.target:
                exit_reason = 'Target Hit'
                buy_id = self.logs.at[buy_index, 'BuyID']
                print(f"Target hit for BuyID {self.logs.at[buy_index, 'BuyID']} at row {buy_index}. Exiting tradeqntry : {qnty}")
                return self.sell_stock(buy_id=buy_id, quantity=qnty, sell_price=current_price, date=time.strftime('%Y-%m-%d'),exit_reason = exit_reason)
    
            # Check if the current price is below the stop loss
            elif current_price <= self.stoploss:
                exit_reason = 'SL Hit'
                buy_id = self.logs.at[buy_index, 'BuyID']
                print(f"Stop Loss hit for BuyID {self.logs.at[buy_index, 'BuyID']} at row {buy_index}. Exiting trade qntry : {qnty}")
                return self.sell_stock(buy_id=buy_id, quantity=qnty, sell_price=current_price, date=time.strftime('%Y-%m-%d'),exit_reason = exit_reason)

        return None



    def auto_exit_with_trailing(self, list_of_symbol):
        self.trade_executed = False
        self.trailing_sl = 10
        self.trailing_target = 15

        def process_symbol(i):
            symbol = i['Symbol']
            current_price = i['Price']

            relevant_entries_ls = self.logs[(self.logs['SLValue'] >= current_price) & (self.logs['Symbol'] == symbol)]

            for buy_index, buy_row in relevant_entries_ls.iterrows():
                buying_price = buy_row['BuyPrice']
                qnty = buy_row['PendingQuantity']
                buy_id = buy_row['BuyID']

                if qnty != 0:
                    if current_price < self.stoploss:
                        percentage_loss = ((buying_price - current_price) / buying_price) * 100
                        self.sell_stock(buy_id=buy_id, quantity=qnty, sell_price=current_price, date=time.strftime('%Y-%m-%d'), exit_reason="Trailing SL")
                        print(f" SL EXIT DONE [{buy_id}] LOSS% : {percentage_loss:.2f}%")

            relevant_entries_target = self.logs[self.logs['Symbol'] == symbol]
            for buy_index, buy_row in relevant_entries_target.iterrows():
                buying_price = buy_row['BuyPrice']
                qnty = buy_row['PendingQuantity']
                buy_id = buy_row['BuyID']

                if current_price > self.target:
                    new_sl = round((self.target - (self.target * (self.trailing_sl / 100))), 2)
                    new_target = round((self.target + (self.target * (self.trailing_target / 100))), 2)

                    self.stoploss = new_sl
                    self.target = new_target
                    percentage_gain = ((current_price - buying_price) / buying_price) * 100

                    # Update SLValue and TargetValue for all relevant entries
                    self.logs.at[buy_index, 'SLValue'] = new_sl
                    self.logs.at[buy_index, 'TargetValue'] = new_target

                    print(f" TRAILING TARGET ACHIEVED [{buy_id}] GAIN% : {percentage_loss:.2f}% ")

        with concurrent.futures.ThreadPoolExecutor() as executor:
            executor.map(process_symbol, list_of_symbol)

    
    def stats(self):
            df = self.logs
            total_trade = len(df.index)
            pnl = df.PnL.sum()
            winners = len(df[df.PnL > 0])
            losers = len(df[df.PnL <= 0])
            win_ratio = round((winners / total_trade) * 100, 2)
    
            # Calculate additional metrics
            capital = self.balance
            max_win = round(df[df.PnL > 0].PnL.max(), 2)
            max_loss = round(df[df.PnL <= 0].PnL.min(), 2)
            total_profit = round(df.PnL.sum(), 2)
            total_profit_percentage = round((total_profit / self.balance) * 100, 2)
    
            # Prepare the data for tabular representation
            parameters = ['Total Trades', 'Capital', 'Total Wins', 'Total Losses', 'Win Ratio',
                          'Max Win', 'Max Loss', 'Total Profit', ' Grow Profit %']
            data_points = [total_trade, capital, winners, losers, f"{win_ratio}%",
                           max_win, max_loss, total_profit, f"{total_profit_percentage}%"]
            data = list(zip(parameters, data_points))

            # Print the tabular representation
            print(tabulate(data, headers=['Parameters', 'Values'], tablefmt='psql'))

In [46]:
trader = StockTrader(initial_balance=100000)

In [47]:
buy_id_1 = trader.buy_stock(quantity=50, price=60, date='2023-01-01',symbol = "IPL")
buy_id_1 = trader.buy_stock(quantity=50, price=60, date='2023-01-01',symbol = "NJ")
buy_id_1 = trader.buy_stock(quantity=50, price=60, date='2023-01-01',symbol = "SMP")
# sell_result_1 = trader.sell_stock(buy_id=buy_id_1, quantity=10, sell_price=500, date='2023-02-01')

In [48]:
trader.logs

,Symbol,Status,BuyID,Quantity,BuyPrice,BuyDatetime,SellTransactions,PendingQuantity,AverageSellPrice,PnL,SLValue,TargetValue,ExitReason
0,IPL,Open,1703520194,50,60,2023-01-01,[],50,0,0,50,80,Manual
1,NJ,Open,1703520195,50,60,2023-01-01,[],50,0,0,50,80,Manual
2,SMP,Open,1703520196,50,60,2023-01-01,[],50,0,0,50,80,Manual


In [49]:
trader.auto_exit_with_trailing([{"Symbol":"SMP","Price":40},{"Symbol":"NJ","Price": 40 },{"Symbol":"IPL","Price": 40 }])

 SL EXIT DONE [1703520196] LOSS% : 33.33%
 SL EXIT DONE [1703520194] LOSS% : 33.33%
 SL EXIT DONE [1703520195] LOSS% : 33.33%


In [50]:
trader.stats()

+---------------+----------+
| Parameters    | Values   |
|---------------+----------|
| Total Trades  | 3        |
| Capital       | 97000    |
| Total Wins    | 0        |
| Total Losses  | 3        |
| Win Ratio     | 0.0%     |
| Max Win       | nan      |
| Max Loss      | -1000    |
| Total Profit  | -3000    |
| Grow Profit % | -3.09%   |
+---------------+----------+
